<a href="https://colab.research.google.com/github/dm-fedorov/advanced-python/blob/master/about_iterator.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory" target="_blank"></a>

Самая важная схема занятия:

![itertools](http://nvie.com/img/relationships.png)

### Containers

Содержит элементы внутри себя, поэтому можем это проверить:

In [ ]:
assert 1 in [1, 2, 3]      # lists

In [ ]:
assert 4 not in [1, 2, 3]

In [ ]:
assert 1 in {1, 2, 3}      # sets

In [ ]:
assert 4 not in {1, 2, 3}

In [ ]:
assert 1 in (1, 2, 3)      # tuples

In [ ]:
assert 4 not in (1, 2, 3)

Словари проверяются на наличие ключей:

In [ ]:
d = {1: 'foo', 2: 'bar', 3: 'qux'}

In [ ]:
assert 1 in d

In [ ]:
assert 4 not in d

In [ ]:
assert 'foo' not in d  # 'foo' is not a _key_ in the dict

Строка может содержать подстроку:

In [ ]:
s = 'foobar'

In [ ]:
assert 'b' in s

In [ ]:
assert 'x' not in s

In [ ]:
assert 'foo' in s  # a string "contains" all its substrings

Не все контейнеры итерабельные, например вероятностный [фильтр Блума](https://ru.wikipedia.org/wiki/%D0%A4%D0%B8%D0%BB%D1%8C%D1%82%D1%80_%D0%91%D0%BB%D1%83%D0%BC%D0%B0).

#### Iterable
Объект, способный возвращать элементы по одному

* у него есть `__iter__()` (`iterator protocol`) или `__getitem__()` (`sequence protocol`)
* вызов `iter()` превращает в итератор
* `for` вызывает `iter()` неявно, сохраняет итератор в безымянной переменной

#### Iterator
Объект, представляющий последовательность данных

* есть `__iter__()`, который возвращает себя же
* `iterator` тоже `iterable`
* оканчивается после `raise StopIteration`
* `iter()` возвращает свежий итератор для `iterable`, но "выдохшийся" - для `iterator'а`

In [ ]:
lst = [1, 2, 3]
type(lst)

In [ ]:
x = iter(lst)
print(type(x))
print(x)

In [ ]:
next(x)

In [ ]:
next(x)

In [ ]:
next(x)

In [ ]:
next(x)

Очевидно, что для списка можно вызвать `for`:

In [ ]:
x = [1, 2, 3]
for elem in x: # это синтаксический сахар для цикла while
    print(elem)

![it](http://nvie.com/img/iterable-vs-iterator.png)

### Каким образом это выглядит с точки зрения Python? 

Здесь скрыты (то есть неявно зарыты) итераторы:

In [ ]:
x = [1, 2, 3]
iterator = iter(x)
# это бесконечный цикл:
while True:
    try:
        item = next(iterator)
    except StopIteration:
        del iterator
        break
    print(item)

Копнем глубже (вспомним, что вызов функции скрывает вызов специального метода класса):

In [ ]:
lst = [1, 2, 3]
iterator = lst.__iter__()
# это бесконечный цикл:
while True:    
    try:
        item = iterator.__next__()
    except StopIteration:
        del iterator
        break
    print(item)

Таким образом, чтобы работал цикл `for` необходимо наличие специальных методов, или говорят о *реализации протокола итератора*.

### Справка
для реализации `Sequence protocol` необходимо реализовать:

* `__getitem__()`. Получает на вход индекс или `slice`, возвращает нужные элемент(ы). Кидает `IndexError`, если нет такого элемента
* `__len__()`. Возвращает длину последовательности

И виртуальная машина Python нам тоже показывает работу `_ITER`:

In [ ]:
import dis
x = [1, 2, 3]
dis.dis('for _ in x: pass')

Создадим собственный класс, который поддерживает через специальные "дандер" методы протокол итератора:

In [ ]:
class Repeater:
    def __init__(self, value):
        self.value = value
    def __iter__(self):
        return self
    def __next__(self):
        return self.value

Теперь у нас есть все, чтобы использовать `Repeater` в цикле `for`:

In [ ]:
repeater = Repeater('Привет')
for item in repeater:
    print(item) 
    break # иначе бесконечный цикл

Итераторы Python не могут быть «обнулены», когда они завершатся. В этом случае им полагается вызывать исключение `StopIteration` при следующем вызове функции `next()`. Чтобы возобновить итерации, нужно запросить свежий объект-итератор при помощи функции `iter()`.

In [ ]:
class BoundedRepeater:
    def __init__(self, value, max_repeats):
        self.value = value
        self.max_repeats = max_repeats
        self.count = 0
    def __iter__(self):
        return self # возвращает объект-итератор, в котором реализован __next__
    def __next__(self):
        if self.count >= self.max_repeats:
            raise StopIteration
        self.count += 1
        return self.value

Итерации прекращаются после ряда повторений, определенных в параметре `max_repeats`:

In [ ]:
repeater = BoundedRepeater('Привет', 3)
for item in repeater:
    print(item)

Если переписать этот последний пример цикла `for...in` , устранив часть *синтаксического сахара*, то в итоге получим следующий фрагмент кода:

In [ ]:
repeater = BoundedRepeater('Привет', 3)
iterator = iter(repeater)
while True:
    try:
        item = next(iterator)
    except StopIteration:
        del iterator
        break
    print(item)

Или перепишем через вызов специальных методов:

In [ ]:
repeater = BoundedRepeater('Привет', 3)
iterator = repeater.__iter__()
while True:
    try:
        item = iterator.__next__()
    except StopIteration:
        del iterator
        break
    print(item)

### Про range

выдает значения по запросу (ленивая функция), но не является итератором:

In [ ]:
for _ in range(3):
    print(_)

In [ ]:
next(range(10))

Можем сделать из `range` итератор:`

In [ ]:
iter_range = iter(range(10))
iter_range

In [ ]:
next(iter_range)

In [ ]:
next(iter_range)

In [ ]:
list(iter_range)

In [ ]:
next(iter_range)

In [ ]:
list(range(10))

### Задание:

In [ ]:
class CompressedList(list):
    def __iter__(self):
        return ComressedListIterator(self)

class ComressedListIterator:
    def __init__(self, arr):
        self.arr = arr
        self.counter = 0
        self.index = 0
  
    def __next__(self):
        # если все пары перебрали, raise StopIteration()
        
        # если не все повторяющиеся элементы 
        # текущей пары self.index выдали - увеличиваем counter
        # выдаем еще один элемент
    
        # если все элементы текущей пары выдали, переходим к 
        # следующей паре

original = [1, 1, 1, 1, 2, 2, 1, 1, 1, 3, 3, 3, 3]
compressed = CompressedList([(1, 4), (2, 2), (1, 3), (3, 4)])

decompressed = [x for x in compressed]

print(original)
print(decompressed)
print(original == decompressed)

Все функции модуля [itertools](https://docs.python.org/3/library/itertools.html) возвращают итераторы. 

Некоторые из них [производят бесконечные последовательности](https://docs.python.org/3/library/itertools.html#itertools.count):

In [ ]:
from itertools import count

In [ ]:
counter = count(start=13)

In [ ]:
next(counter)

In [ ]:
next(counter)

Некоторые производят [бесконечные последовательности из конечных последовательностей](https://docs.python.org/3/library/itertools.html#itertools.cycle):

In [ ]:
from itertools import cycle

In [ ]:
colors = cycle(['red', 'white', 'blue'])

In [ ]:
next(colors)

In [ ]:
next(colors)

In [ ]:
next(colors)

In [ ]:
next(colors)

Некоторые производят [конечные последовательности из бесконечных последовательностей](https://docs.python.org/3/library/itertools.html#itertools.islice):

In [ ]:
from itertools import islice

In [ ]:
colors = cycle(['red', 'white', 'blue'])  # infinite

In [ ]:
limited = islice(colors, 0, 4)            # finite

In [ ]:
for x in limited:                         # so safe to use for-loop on
    print(x)

Давайте создадим итератор, производящий [числа Фибоначчи](https://ru.wikipedia.org/wiki/%D0%A7%D0%B8%D1%81%D0%BB%D0%B0_%D0%A4%D0%B8%D0%B1%D0%BE%D0%BD%D0%B0%D1%87%D1%87%D0%B8):

In [ ]:
from itertools import islice

class fib:
    def __init__(self):
        self.prev = 0
        self.curr = 1

    def __iter__(self):
        return self

    def __next__(self):
        value = self.curr
        self.curr += self.prev
        self.prev = value
        return value

f = fib()
list(islice(f, 0, 10))

"Ленивый" итератор не станет работать, пока его об этом не попросят.

### Generators - особый тип итератора

В Python есть два типа генераторов: *функции* и *выражения*. 

Функция-генератор - это любая функция, в теле которой встречается ключевое слово `yield`.

Можно переписать класс `Repeater` в виде генератора:

In [ ]:
def repeater(value):
    while True:
        yield value

В [PEP 255](https://www.python.org/dev/peps/pep-0255/) есть обсуждение, почему Гвидо решил оставить ключевое слово `def` для генераторов.

Генераторы похожи на обычные функции, но вместо инструкции возврата `return` в них для передачи данных назад источнику вызова используется инструкция `yield`.

In [ ]:
for x in repeater('Привет'):
    print(x)
    break # иначе бесконечный цикл

Начнем с того, что вызов функции-генератора вообще не выполняет функцию. 

*Он просто создает и возвращает объект-генератор*:

In [ ]:
repeater('Эй')

Программный код в функции-генераторе исполняется только тогда, когда функция `next()` вызывается с объектом-генератором в качестве аргумента:

In [ ]:
generator_obj = repeater('Эй')
next(generator_obj)

In [ ]:
next(generator_obj)

Это типичный итератор:

In [ ]:
from itertools import islice

generator_obj = repeater('Эй')
list(islice(generator_obj, 0, 10))

Если вы еще раз прочитаете код функции `repeater`, то увидите, что, судя по всему, ключевое слово `yield` каким-то образом останавливает эту функцию-генератор посередине исполнения, а затем возобновляет ее на более позднем этапе:

In [ ]:
def repeater(value):
    while True:
        yield value

И это вполне подходящая модель того, что здесь происходит. Дело в том, что, когда инструкция `return` вызывается внутри функции, она безвозвратно передает управление назад источнику вызова функции. Когда же вызывается инструкция `yield`, она тоже передает управление назад источнику вызова функции — но она это делает лишь временно.

В отличие от инструкции `return`, которая избавляется от локального состояния функции, инструкция `yield` приостанавливает функцию и сохраняет ее локальное состояние. На практике это означает, что локальные переменные и состояние исполнения функции-генератора лишь откладываются в сторону и не выбрасываются полностью. 

Исполнение может быть возобновлено в любое время вызовом функции `next()` с генератором в качестве аргумента:

In [ ]:
iterator = repeater('Привет')
next(iterator)

In [ ]:
next(iterator)

Напомним, что в нашем итераторе на основе класса мы смогли подать сигнал об окончании итераций путем вызова исключения `StopIteration` вручную. Поскольку генераторы полностью совместимы с итераторами на основе класса, за сценой будет по-прежнему происходить то же самое. К счастью, на этот раз мы будем работать с более приятным интерфейсом. Генераторы прекращают порождать значения, как только поток управления возвращается из функции-генератора каким-либо иным способом, кроме инструкции `yield`. Это означает, что вам больше вообще не нужно заботиться о вызове исключения `StopIteration`!

Приведу пример:

In [ ]:
def repeat_three_times(value):
    print('start')
    yield value
    print('continue')
    yield value
    print('end')
    yield value    

Обратите внимание: эта функция-генератор не содержит никакого цикла. В действительности она проста как божий день и состоит всего из трех инструкций `yield`. Если `yield` временно приостанавливает выполнение функции и передает значение назад источнику вызова, то что произойдет, когда мы достигнем конца этого генератора? 

Давайте узнаем:

In [ ]:
for x in repeat_three_times('Всем привет'):
    print('-->', x)

Как вы, возможно, и ожидали, этот генератор прекратил порождать новые значения после трех итераций.

Можно предположить, что он это сделал путем вызова исключения `StopIteration`, когда исполнение достигло конца функции. Но чтобы быть до конца уверенными, давайте подтвердим это еще одним экспериментом:

In [ ]:
iterator = repeat_three_times('Всем привет')
next(iterator)

In [ ]:
next(iterator)

In [ ]:
next(iterator)

In [ ]:
next(iterator)

Этот итератор вел себя именно так, как мы и ожидали. Как только мы достигаем конца функции-генератора, он начинает вызывать `StopIteration`, сигнализируя о том, что у него больше нет значений, которые он мог бы предоставить.

Класс `BoundedIterator` реализовал итератор, который будет повторять значение, заданное определенное количество раз.

Почему бы не попробовать реализовать класс `BoundedRepeater` заново как функцию-генератор? 

Сделаю первую попытку:

In [ ]:
def bounded_repeater(value, max_repeats):
    count = 0
    while True:
        if count >= max_repeats:
            return
        count += 1
        yield value

Я преднамеренно сделал цикл `while` в этой функции несколько громоздким. Я хотел продемонстрировать, как вызов инструкции `return` из генератора приводит к остановке итераций с исключением `StopIteration`. 

Мы вскоре подчистим и еще немного упростим эту функцию-генератор, но сначала давайте испытаем то, что у нас есть сейчас:

In [ ]:
for x in bounded_repeater('Привет', 4):
    print(x)

Великолепно! Теперь у нас есть генератор, который прекращает порождать значения после настраиваемого количества повторений. Он использует инструкцию `yield`, чтобы передавать значения назад до тех пор, пока он наконец не натолкнется на инструкцию `return` и итерации не прекратятся.

Мы можем упростить этот генератор еще больше. Воспользуемся тем, что в конец каждой функции Python добавляет неявную инструкцию `return None`. 

И вот как будет выглядеть наша окончательная реализация:

In [ ]:
def bounded_repeater(value, max_repeats):
    for i in range(max_repeats):
        yield value

Генераторы помогают «абстрагироваться от» большей части шаблонного кода, который в других обстоятельствах был бы необходим во время написания итераторов на основе класса.

Последовательность Фибоначчи, реализованная как генератор:

In [ ]:
from itertools import islice

def fib():
    prev, curr = 0, 1
    while True:
        yield curr
        prev, curr = curr, prev + curr

f = fib()
list(islice(f, 0, 10))

### Задание с использованием функции-генератора

Создать класс для генерации ограниченной арифметичесой прогрессии чисел произвольного типа:

```Python
>>> ap = ArithProgress(0, 1, 3)
>>> list(ap)
[0, 1, 2]
>>> ap = ArithProgress(1, .5, 3)
>>> list(ap)
[1.0, 1.5, 2.0, 2.5]
```

### Итерирование в обратном порядке:

In [ ]:
class Countdown:
    def __init__(self, start):
        self.start = start
    
    # Прямой итератор
    def __iter__(self):
        n = self.start
        while n > 0:
            yield n
            n -= 1
    
    # Обратный итератор
    def __reversed__(self):
        n = 1
        while n <= self.start:
            yield n
            n += 1

In [ ]:
c = iter(Countdown(4))
c.__next__()

In [ ]:
c = reversed(Countdown(4))
c.__next__()

In [ ]:
c.__next__()

Генераторы можно увидеть в следующем коде:

```Python
def something():
    result = []
    for ... in ...:
        result.append(x)
    return result
```

Заменить на следующий код:
```Python
def iter_something():
    for ... in ...:
        yield x

# def something():  # Only if you really need a list structure
#     return list(iter_something())
```

### Задание:

In [ ]:
class CompressedList(list):
    def __iter__(self):
        return ComressedListIterator(self)

class ComressedListIterator:
    def __init__(self, arr):
        self.arr = arr
        self.counter = 0
        self.index = 0
  
    def __next__(self):
        # если все пары перебрали, raise StopIteration()
        
        # если не все повторяющиеся элементы 
        # текущей пары self.index выдали - увеличиваем counter
        # выдаем еще один элемент
    
        # если все элементы текущей пары выдали, переходим к 
        # следующей паре

original = [1, 1, 1, 1, 2, 2, 1, 1, 1, 3, 3, 3, 3]
compressed = CompressedList([(1, 4), (2, 2), (1, 3), (3, 4)])

decompressed = [x for x in compressed]

print(original)
print(decompressed)
print(original == decompressed)

Генераторы другого типа похожи на `list comprehension` (пер. *списочное встраивание*).

Это список:

In [ ]:
numbers = [1, 2, 3, 4, 5, 6]
[x * x for x in numbers]

Множество (a set comprehension):

In [ ]:
{x * x for x in numbers}

Словарь (a dict comprehension):

In [ ]:
{x: x * x for x in numbers}

Выражение генератор (это **НЕ** tuple comprehension):

In [ ]:
lazy_squares = (x * x for x in numbers)

In [ ]:
lazy_squares

In [ ]:
next(lazy_squares)

In [ ]:
list(lazy_squares)

In [ ]:
next(lazy_squares)

In [ ]:
gen = (x * x for x in range(2328346283764826348726347628374628376482763482763482634876))
print(next(gen))
print(next(gen))
print(next(gen))
# отложенные вычисления, т.е. один элемент по запросу

Выражения-генераторы представляют собой еще более эффективную краткую форму для создания итераторов. Благодаря простому и сжатому синтаксису, который похож на конструкцию включения в список, вы сможете определять итераторы в одной строке кода.

Приведу пример:

In [ ]:
iterator = ('Привет' for i in range(3))

Во время выполнения итераций данное выражение-генератор порождает ту же самую последовательность значений, что и функция-генератор `bounded_repeater`, которую мы написали ранее.

Разве не удивительно, что однострочное выражение-генератор теперь делает работу, для выполнения которой ранее требовалась четырехстрочная функция-генератор или намного более длинный итератор на основе класса?

Привет, синтаксический сахар!

In [ ]:
iterator = ('Привет' for i in range(3))
for x in iterator:
    print(x)

Из нашего однострочного выражения-генератора мы получили те же самые результаты, которые мы получали из функции-генератора `bounded_repeater`.

Правда, есть одно маленькое предостережение: после того как выражение-генератор было использовано, оно не может быть перезапущено или использовано снова. Поэтому в некоторых случаях предпочтительнее использовать функции-генераторы или итераторы на основе класса.

Выражения-генераторы несколько напоминают включения в список:

In [ ]:
listcomp = ['Привет' for i in range(3)]
genexpr = ('Привет' for i in range(3))

Однако в отличие от включений в список выражения-генераторы не конструируют объекты-списки. Вместо этого они генерируют значения «точно в срок» подобно тому, как это сделал бы итератор на основе класса или функция-генератор.

Присваивая выражение-генератор переменной, вы просто получите итерируемый «объект-генератор»:

In [ ]:
listcomp

In [ ]:
genexpr

Для того чтобы получить доступ к значениям, порожденным выражением-генератором, вам нужно вызвать с ним метод `next()` точно так же, как вы бы сделали с любым другим итератором:

In [ ]:
next(genexpr)

Как вариант, вы также можете вызвать функцию `list()` c выражением-генератором, в результате чего вы сконструируете объект-список, содержащий все произведенные значения:

In [ ]:
genexpr = ('Привет' for i in range(3))
list(genexpr)

Разумеется, это был всего лишь игрушечный пример, который показывает, как можно «преобразовывать» выражение-генератор (или любой другой итератор, если уж на то пошло) в список. Если же вам нужен объект-список прямо на месте, то в большинстве случаев вы с самого начала просто пишете включение в список.

Шаблон, который вы должны увидеть, выглядит следующим образом:

```
genexpr = (expression for item in collection)
```

Приведенный выше «образец» выражения-генератора соответствует следующей ниже функции-генератору:

In [ ]:
def generator():
    for item in collection:
        yield expression

Точно так же, как и с включением в список, он дает вам типовой шаблон в стиле «формы для печенья», который можно применять ко многим функциям-генераторам с целью их преобразования в сжатые выражения-генераторы.

В этот шаблон можно добавить еще одно полезное дополнение, и это фильтрация элемента по условиям. Приведем пример:

In [ ]:
even_squares = (x * x for x in range(10) if x % 2 == 0)

Данный генератор порождает квадрат всех четных целых чисел от нуля до девяти. Фильтрующее условие с использованием оператора остатка % отклонит любое значение, которое не делится на два:

In [ ]:
for x in even_squares:
    print(x)

Давайте обновим наш шаблон выражения-генератора. После добавления фильтрации элементов посредством условия `if` шаблон выглядит так:

```
genexpr = (expression for item in collection if condition)
```

И снова этот шаблон соответствует относительно прямолинейной, но более длинной функции-генератору. Синтаксический сахар в своих лучших проявлениях:

In [ ]:
def generator():
    for item in collection:
        if condition:
            yield expression

Поскольку выражения-генераторы являются, скажем так, выражениями, вы можете их использовать в одной строке вместе с другими инструкциями. 

Например, вы можете определить итератор и употребить его прямо на месте при помощи цикла `for`:

In [ ]:
for x in ('Buongiorno' for i in range(3)):
    print(x)

Есть и другой синтаксический трюк, который можно использовать для того, чтобы сделать выражения-генераторы красивее. Круглые скобки, окружающие выражение-генератор, могут быть опущены, если выражение-генератор используется в качестве единственного аргумента функции:

In [ ]:
sum((x * 2 for x in range(10)))

In [ ]:
# Сравните с:
sum(x * 2 for x in range(10))

Это позволяет писать сжатый и высокопроизводительный код. Поскольку выражения-генераторы генерируют значения «точно в срок» подобно тому, как это делает итератор на основе класса или функция-генератор, они эффективно используют оперативную память.

Выражения-генераторы являются полезным питоновским инструментом в вашем наборе, но это не значит, что они должны использоваться для решения каждой задачи, с которой вы сталкиваетесь. В случае составных итераторов часто лучше написать функцию-генератор или даже итератор на основе класса.

Например, можно определить следующий ниже генератор, который производит серию целочисленных значений от одного до восьми, поддерживая нарастающий счетчик и выдавая новое значение всякий раз, когда с ним вызывается функция `next()`:

In [ ]:
def integers():
    for i in range(1, 9):
        yield i

Вы можете подтвердить такое поведение:

In [ ]:
chain = integers()
list(chain)

Пока что не очень интересно. Но сейчас мы быстро это изменим. Дело в том, что генераторы могут быть «присоединены» друг к другу, благодаря чему можно строить эффективные алгоритмы обработки данных, которые работают как *конвейер*. 

Вы можете взять «поток» значений, выходящих из генератора `integers()`, и направить их в еще один генератор. Например, такой, который принимает каждое число, возводит его в квадрат, а затем передает его дальше:

In [ ]:
def squared(seq):
    for i in seq:
        yield i * i

Это похоже на то, как работают [конвейеры в UNIX](https://ru.wikipedia.org/wiki/%D0%9A%D0%BE%D0%BD%D0%B2%D0%B5%D0%B9%D0%B5%D1%80_(Unix)). Мы состыковываем последовательность процессов в цепочку так, чтобы результат каждого процесса подавался непосредственно на вход следующего. Почему бы в наш конвейер не добавить еще один шаг, который инвертирует каждое значение, а потом передает его на следующий шаг обработки в цепи:

In [ ]:
def negated(seq):
    for i in seq:
        yield -i

Если мы перестроим нашу цепочку генераторов и добавим `negated` в конец, то вот что мы получим на выходе:

In [ ]:
chain = negated(squared(integers()))
list(chain)

Моя любимая фишка формирования цепочки генераторов состоит в том, что обработка данных происходит по одному элементу за один раз. Буферизация между шагами обработки в цепочке отсутствует:

1. Генератор `integers` выдает одно-единственное значение, скажем, 3.
2. Это значение «активирует» генератор `squared`, который обрабатывает значение и передает его на следующую стадию как 3 × 3 = 9.
3. Квадрат целого числа, выданный генератором `squared`, немедленно передается в генератор `negated`, который модифицирует его в –9 и выдает его снова.

Вы можете продолжать расширять эту цепочку генераторов, чтобы отстроить конвейер обработки со многими шагами. Он по-прежнему будет выполняться эффективно и может легко быть модифицирован, потому что каждым шагом в цепочке является отдельная функция-генератор.

Каждая отдельная функция-генератор в этом конвейере обработки довольно сжатая. С помощью небольшой уловки мы можем сжать определение этого конвейера еще больше, не сильно жертвуя удобочитаемостью:

In [ ]:
integers = range(8)
squared = (i * i for i in integers)
negated = (-i for i in squared)

Обратите внимание, как я заменил каждый шаг обработки в цепочке на выражение-генератор, строящийся на выходе из предыдущего шага. 

Этот программный код эквивалентен цепочке генераторов, которые мы построили в этом разделе выше:

In [ ]:
negated

In [ ]:
list(negated)

Единственным недостатком применения выражений-генераторов является то, что их не получится сконфигурировать с использованием аргументов функции и вы не сможете повторно использовать то же самое выражение-генератор многократно в том же самом конвейере обработки.

Но, безусловно, во время сборки конвейеров вы можете свободно комбинировать выражения-генераторы и обычные генераторы на свой вкус.

В случае с составными конвейерами это поможет улучшить удобочитаемость.

### Выражения-генераторы для работы с большими файлами

In [ ]:
!cat data/text.txt

Проблема в том, что `списковое включение` создает весь список целиком. Это допустимо для небольшого объема, но мы же хотим в будущем анализировать big data? 

In [ ]:
value = [len(x) for x in open('data/text.txt')]
value

В качестве решения поможет *выражение-генератор*, которое не загружает в память последовательность целиком. Вместо это вычисляется итератор, который изначально не выполняет никаких действий:  

In [ ]:
it = (len(x) for x in open('data/text.txt'))
it

С помощью встроенной функции `next` можно на шаг продвинуться по мере необходимости:

In [ ]:
next(it)

In [ ]:
next(it)

In [ ]:
next(it)

Выражения-генераторы можно объединять, создавая "эффект домино":

In [ ]:
roots = ((x, x**2) for x in it)
roots

In [ ]:
next(roots)

### Литература:

    - https://nvie.com/posts/iterators-vs-generators/
    - https://treyhunner.com/2016/12/python-iterator-protocol-how-for-loops-work/    
    - https://treyhunner.com/2018/02/python-range-is-not-an-iterator/
    - Лекция 8 в CS. Итераторы (Программирование на Python): https://www.youtube.com/watch?v=Xxuy1zFCMhc